In [1]:
from __future__ import annotations
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from torch.distributions.normal import Normal
import pickle
import gymnasium as gym
from segway_sim.envs import SegwayEnv

In [2]:
class PID:
    def __init__(self, obs_space_dims: int, action_space_dims: int, nn_file_path: str = None):
        return
    def sample_action(self, obs: np.ndarray) -> float:
        angel = obs[0]
        # print(angel)
        ac = angel * -2
        return [ac]

    def update(self):
        return
    def save(self, nn_file_path: str):
        return

env = SegwayEnv(max_ep_len = 1000)
wrapped_env = gym.wrappers.RecordEpisodeStatistics(env, 50)  # Records episode-reward
# wrapped_env = gym.wrappers.RecordVideo(wrapped_env, 'videos', episode_trigger = lambda x: x % 1000 == 0)

obs_space_dims = env.observation_space.shape[0]
action_space_dims = env.action_space.shape[0]
print('ob space:', obs_space_dims)
print('ac space:', action_space_dims)

seed = 1
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

agent = PID(obs_space_dims, action_space_dims,nn_file_path = 'nn_segway.pkl')
reward_over_episodes = []

for episode in range(1):
    obs, info = wrapped_env.reset(seed=seed)
    done = False
    while not done:
        action = agent.sample_action(obs)
        obs, reward, terminated, truncated, info = wrapped_env.step(action)
        done = terminated or truncated

    reward_over_episodes.append(wrapped_env.return_queue[-1])
    agent.update()

    if episode % 1000 == 0:
        avg_reward = int(np.mean(wrapped_env.return_queue))
        print("Episode:", episode, "Average Reward:", avg_reward)

ob space: 3
ac space: 1
Episode: 0 Average Reward: 998


In [10]:
from gymnasium.wrappers import RecordVideo
# env = gym.make("InvertedPendulum-v4", render_mode = 'rgb_array')
acc_y = []
acc_z = []
for i in range(1000):
    env = SegwayEnv(max_ep_len = 1000, render_mode="rgb_array")
    video_env = RecordVideo(env, video_folder="videos", name_prefix="Segway")
    observation, info = video_env.reset(seed=seed)
    #print(observation)
    for i in range(100):
        action = agent.sample_action(observation)
        # action = env.action_space.sample()
        observation, reward, terminated, truncated, info = video_env.step(action)
        if i == 99:
            acc_y.append(observation[0])
            acc_z.append(observation[1])
        if terminated or truncated:
            print("terminated at", i)
            break
            observation, info = video_env.reset()
    #video_env.close()
np.savetxt("acc_x1.txt", acc_y)
np.savetxt("acc_y1.txt", acc_z)

KeyboardInterrupt: 